In [9]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

In [10]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [11]:
class Analizer:
    def __init__(self):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
    
    def has_minimum_requirements(self, df, sort_by="r2", boundary=0.75):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [12]:
analize = Analizer()
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_test,r2_val,r2_vt,mse,mse_test,mse_val,mse_vt,Architecture
0,model_15_9_7,0.996414,0.994509,0.989020,0.992938,0.003586,0.007529,0.009605,0.008506,"Hidden Size=[40], regularizer=0.01, learning_r..."
1,model_15_9_6,0.996394,0.994712,0.990386,0.993527,0.003606,0.007251,0.008411,0.007797,"Hidden Size=[40], regularizer=0.01, learning_r..."
2,model_15_9_8,0.996383,0.994268,0.987672,0.992332,0.003617,0.007860,0.010785,0.009236,"Hidden Size=[40], regularizer=0.01, learning_r..."
3,model_15_9_9,0.996315,0.994004,0.986359,0.991724,0.003685,0.008223,0.011933,0.009969,"Hidden Size=[40], regularizer=0.01, learning_r..."
4,model_15_9_5,0.996306,0.994858,0.991743,0.994079,0.003694,0.007051,0.007223,0.007132,"Hidden Size=[40], regularizer=0.01, learning_r..."
...,...,...,...,...,...,...,...,...,...,...
369,model_5_7_0,0.977120,0.982933,0.986503,0.985547,0.022880,0.011935,0.013545,0.012692,"Hidden Size=[15], regularizer=0.01, learning_r..."
381,model_12_6_6,0.976908,0.958143,0.947449,0.955359,0.023092,0.054264,0.052403,0.053388,"Hidden Size=[30], regularizer=0.1, learning_ra..."
389,model_12_6_5,0.976765,0.959745,0.955447,0.959417,0.023235,0.052188,0.044428,0.048536,"Hidden Size=[30], regularizer=0.1, learning_ra..."
415,model_12_6_4,0.976283,0.961396,0.963662,0.963588,0.023717,0.050047,0.036236,0.043548,"Hidden Size=[30], regularizer=0.1, learning_ra..."
